# OCR Matching

In this notebook we want use OCR on pdf and jpg for extracting information and matching this with a dataset

In [1]:
pip install  pypdf langchain tika typing fuzzywuzzy pymupdf easyocr

Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\Users\Simone\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.


In [2]:
import pandas as pd
from langchain.document_loaders import PyPDFLoader
from tika import parser
from typing import List
import re
from typing import List, Tuple
from fuzzywuzzy import fuzz, process
import fitz
import easyocr
import numpy as np


c:\Users\Simone\AppData\Local\Programs\Python\Python310\lib\site-packages\fuzzywuzzy\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [3]:
dataset = pd.read_csv('https://raw.githubusercontent.com/SimoneFarallo/public_and_social_services/main/data/prest_amb_clean.csv')
dataset.head()

,COD_ASL,ASL,COD_ENTE,ENTE,COD_STRUT,STRUT,TIPO_RAP_SSN,INDIRIZZO,COMUNE_STRUT,PROV_STRUT,TIPO_PREST,COD_BRANCA_MINSLE,BRANCA_MINSLE,COD_BRANCA_REGLE,BRANCA_REGLE,COD_PREST_AMBLE,N_PREST
0,321,ATS DELLA CITTA' METROPOLITANA DI MILANO,705,ASST OVEST MILANESE,2340,POLIAMBULATORIALE PARABIAGO,PUBBLICO ACCREDITATO,VIA XI FEBBRAIO N. 31,PARABIAGO,MI,LABORATORIO,"LABORATORIO ANALISI CHIMICO-CLINICHE, MICROBIO...",112,CHIMICA CLINICA,90115,CALCITONINA,260.1
1,323,ATS DELLA MONTAGNA,713,ASST DELLA VALTELLINA E DELL'ALTO LARIO,2941,POLIAMBULATORIO OSP. MORBEGNO,PUBBLICO ACCREDITATO,VIA MORELLI 1,MORBEGNO,SO,ATT. CLINICA,ORTOPEDIA E TRAUMATOLOGIA,191,ORTOPEDIA E TRAUMATOLOGIA,9346,ALTRE TRAZIONI CUTANEE DEGLI ARTI,162.4
2,327,ATS DELLA VAL PADANA,724,ASST DI CREMONA,3397,POLIAMB. A.O.IST.OSPIT.CREMONA,PUBBLICO ACCREDITATO,VIA LARGO PRIORI N.1,CREMONA,CR,LABORATORIO,"LABORATORIO ANALISI CHIMICO-CLINICHE, MICROBIO...",114,MICROBIOLOGIA-VIROLOGIA,91085,STREPTOCOCCO ANTICORPI ANTI ANTISTREPTOLISINA-...,2627.4
3,328,ATS DI PAVIA,938,FOND. CLINICA DEL LAVORO - PAVIA,2112,POLIAMB. IRCCS MAUGERI,PRIVATO ACCREDITATO,"VIA MAUGERI, 10",PAVIA,PV,ATT. CLINICA,PNEUMOLOGIA,221,PNEUMOLOGIA,89655,MONITORAGGIO INCRUENTO DELLA SATURAZIONE ARTER...,532.0
4,321,ATS DELLA CITTA' METROPOLITANA DI MILANO,707,ASST NORD MILANO,30058,OSPEDALE BASSINI - CINISELLO BALSAMO,PUBBLICO ACCREDITATO,VIA GORKI 50,CINISELLO BALSAMO,MI,LABORATORIO,"LABORATORIO ANALISI CHIMICO-CLINICHE, MICROBIO...",114,MICROBIOLOGIA-VIROLOGIA,90557,TEST ANTIGENICI (RAPID DIAGNOSTICS TEST Ag-RDT...,40.0


## OCR Matching per pdf

In [4]:
#Text ci metti un pdf
def extract_data(text):
    pattern = r'\((\w+)\) - ([A-Z ]+)'
    data = re.findall(pattern, text)
    for i in range(len(data)):
        num = data[i][0]
        num = re.sub(r'^0+', '', num)  # rimuovi gli zeri iniziali
        num = re.sub(r'\D', '', num)  # rimuovi le lettere
        data[i] = (num, data[i][1])
    return data

In [5]:
def match_recipe_to_dataset_pdf(pdf_file: str, dataset: pd.DataFrame, threshold: int = 90) -> List[Tuple]:
    if not isinstance(pdf_file, str):
        raise TypeError("Il percorso del file PDF deve essere una stringa.")
    if not isinstance(dataset, pd.DataFrame):
        raise TypeError("Il dataset deve essere un oggetto pandas.DataFrame.")

    # estraiamo il testo dal file pdf
    pdf_file = fitz.open(pdf_file)
    text = ''
    for page in pdf_file:
        text += page.get_text()

    # individuiamo le parole di interesse
    words = re.findall(r' - ([A-Za-z\s]+)', text) #Funzione regex per estrarre parole dalla ricetta (iniziale =' - ([A-Z]+))'

    # matchiamo le parole con le parole della colonna 'COD_PREST_AMBLE' con fuzzywuzzy
    matches = []
    for word in words:
        if not isinstance(word, str):
            raise TypeError("La parola di interesse deve essere una stringa.") #funzione per errori
        scores = process.extract(word, dataset['COD_PREST_AMBLE'], scorer=fuzz.token_set_ratio, limit=1) #qui faccio fuzzymatch devo capire il parametro limit
        filtered_scores = [score for score in scores if score[1] >= threshold]
        for score in filtered_scores:
            rows = dataset[dataset['COD_PREST_AMBLE'] == score[0]].iterrows() #Decidiamo l'output 
            matches.extend([(word, row[1], score[1]) for row in rows])

    # ordiniamo i risultati in base allo score
    matches_sorted = sorted(matches, key=lambda x: x[2], reverse=True)

    return matches_sorted

In [6]:
cd C:\Users\Simone\Documents\Desktop\public_and_social_services\ricette


C:\Users\Simone\Documents\Desktop\public_and_social_services\ricette


In [7]:
match_recipe_to_dataset_pdf('ricetta_3.pdf',dataset)

[('EMOCROMO',
  COD_ASL                                                            327
  ASL                                               ATS DELLA VAL PADANA
  COD_ENTE                                                           725
  ENTE                                                   ASST DI MANTOVA
  COD_STRUT                                                        30239
  STRUT                          OSP.CIVILE DESTRA SECCHIA-PIEVE CORIANO
  TIPO_RAP_SSN                                      PUBBLICO ACCREDITATO
  INDIRIZZO                                               VIA BUGATTE, 1
  COMUNE_STRUT                                           BORGO MANTOVANO
  PROV_STRUT                                                          MN
  TIPO_PREST                                                 LABORATORIO
  COD_BRANCA_MINSLE    LABORATORIO ANALISI CHIMICO-CLINICHE, MICROBIO...
  BRANCA_MINSLE                                                      113
  COD_BRANCA_REGLE                   

## OCR Matching per png

In [8]:
reader = easyocr.Reader(['it'])

CUDA not available - defaulting to CPU. Note: This module is much faster with a GPU.


In [9]:
def match_recipe_to_dataset_imm(file: str, dataset: pd.DataFrame, threshold: int = 90) -> List[Tuple]:
    if not isinstance(file, str):
        raise TypeError("Il percorso del file PDF deve essere una stringa.")
    if not isinstance(dataset, pd.DataFrame):
        raise TypeError("Il dataset deve essere un oggetto pandas.DataFrame.")
    
    ricetta = reader.readtext(file)
    result_string = ""
    for i in range(len(ricetta)):
        elemento = ricetta[i][1] # stringhe si trovano in questa posizione
        result_string += elemento
        result_string += " "
   
    result_string = result_string.replace(".","") #rimozione punti 
    regex = r"\((\d\w*)\)([A-Z ]+)"
    matches = re.findall(regex, result_string)
    output = []
    for match in matches:
      output.append(match[1])

      #individuiamo le parole di interesse
    words = output 

    # matchiamo le parole con le parole della colonna 'COD_PREST_AMBLE' con fuzzywuzzy
    matches = []
    for word in words:
        if not isinstance(word, str):
            raise TypeError("La parola di interesse deve essere una stringa.") #funzione per errori
        scores = process.extract(word, dataset['COD_PREST_AMBLE'], scorer=fuzz.token_set_ratio, limit=1) #qui faccio fuzzymatch devo capire il parametro limit
        filtered_scores = [score for score in scores if score[1] >= threshold]
        for score in filtered_scores:
            rows = dataset[dataset['COD_PREST_AMBLE'] == score[0]].iterrows() #Decidiamo l'output 
            matches.extend([(word, row[1], score[1]) for row in rows])

    # ordiniamo i risultati in base allo score
    matches_sorted = sorted(matches, key=lambda x: x[2], reverse=True)

    return matches_sorted

In [10]:
match_recipe_to_dataset_imm('ricetta_3.png', dataset)


[(' EMOCROMO HB',
  COD_ASL                                                            327
  ASL                                               ATS DELLA VAL PADANA
  COD_ENTE                                                           725
  ENTE                                                   ASST DI MANTOVA
  COD_STRUT                                                        30239
  STRUT                          OSP.CIVILE DESTRA SECCHIA-PIEVE CORIANO
  TIPO_RAP_SSN                                      PUBBLICO ACCREDITATO
  INDIRIZZO                                               VIA BUGATTE, 1
  COMUNE_STRUT                                           BORGO MANTOVANO
  PROV_STRUT                                                          MN
  TIPO_PREST                                                 LABORATORIO
  COD_BRANCA_MINSLE    LABORATORIO ANALISI CHIMICO-CLINICHE, MICROBIO...
  BRANCA_MINSLE                                                      113
  COD_BRANCA_REGLE               

## Merged OCR Matchnng

In [11]:
def match_recipe_to_dataset(file: str, dataset: pd.DataFrame, threshold: int = 90) -> List[Tuple]:
    if not isinstance(file, str):
        raise TypeError("Il percorso del file deve essere una stringa.")
    if not isinstance(dataset, pd.DataFrame):
        raise TypeError("Il dataset deve essere un oggetto pandas.DataFrame.")

    if file.endswith(".pdf"):
        # estraiamo il testo dal file pdf
        pdf_file = fitz.open(file)
        text = ''
        for page in pdf_file:
            text += page.get_text()

        # individuiamo le parole di interesse
        words = re.findall(r' - ([A-Za-z\s]+)', text)

    elif file.endswith(".png"):
        ricetta = reader.readtext(file)
        result_string = ""
        for i in range(len(ricetta)):
            elemento = ricetta[i][1] # stringhe si trovano in questa posizione
            result_string += elemento
            result_string += " "
       
        result_string = result_string.replace(".","") #rimozione punti 
        regex = r"\((\d\w*)\)([A-Z ]+)"
        matches = re.findall(regex, result_string)
        output = []
        for match in matches:
            output.append(match[1])

        words = output

    else:
        raise ValueError("Il formato del file non è supportato. Inserire un file pdf o png.")

    # matchiamo le parole con le parole della colonna 'COD_PREST_AMBLE' con fuzzywuzzy
    matches = []
    for word in words:
        if not isinstance(word, str):
            raise TypeError("La parola di interesse deve essere una stringa.")
        scores = process.extract(word, dataset['COD_PREST_AMBLE'], scorer=fuzz.token_set_ratio, limit=1)
        filtered_scores = [score for score in scores if score[1] >= threshold]
        for score in filtered_scores:
            rows = dataset[dataset['COD_PREST_AMBLE'] == score[0]].iterrows()
            matches.extend([(word,row[1], score[1]) for row in rows])

    # ordiniamo i risultati in base allo score
    matches_sorted = sorted(matches, key=lambda x: x[2], reverse=True)
    print(len(matches_sorted))

    return matches_sorted, 


In [14]:
#(f"Parola ricercata: '{word}'Informazioni : {row}, score_fuzzy: {score[1]}, numero_ambulatori: {num_rows}")
match_recipe_to_dataset('ricetta_3.png',dataset)

331


([(' EMOCROMO HB',
   COD_ASL                                                            327
   ASL                                               ATS DELLA VAL PADANA
   COD_ENTE                                                           725
   ENTE                                                   ASST DI MANTOVA
   COD_STRUT                                                        30239
   STRUT                          OSP.CIVILE DESTRA SECCHIA-PIEVE CORIANO
   TIPO_RAP_SSN                                      PUBBLICO ACCREDITATO
   INDIRIZZO                                               VIA BUGATTE, 1
   COMUNE_STRUT                                           BORGO MANTOVANO
   PROV_STRUT                                                          MN
   TIPO_PREST                                                 LABORATORIO
   COD_BRANCA_MINSLE    LABORATORIO ANALISI CHIMICO-CLINICHE, MICROBIO...
   BRANCA_MINSLE                                                      113
   COD_BRANCA_REGLE

In [15]:
result_ricetta = match_recipe_to_dataset('ricetta_3.png', dataset)
len(result_ricetta)

331


1

# From OCR to chatbot

In [35]:
def match_recipe_to_query(file: str, dataset: pd.DataFrame, threshold: int = 90) -> List[Tuple]:
    if not isinstance(file, str):
        raise TypeError("Il percorso del file deve essere una stringa.")
    if not isinstance(dataset, pd.DataFrame):
        raise TypeError("Il dataset deve essere un oggetto pandas.DataFrame.")

    if file.endswith(".pdf"):
        # estraiamo il testo dal file pdf
        pdf_file = fitz.open(file)
        text = ''
        for page in pdf_file:
            text += page.get_text()

        # individuiamo le parole di interesse
        words = re.findall(r' - ([A-Za-z\s]+)', text)

    elif file.endswith(".png"):
        ricetta = reader.readtext(file)
        result_string = ""
        for i in range(len(ricetta)):
            elemento = ricetta[i][1] # stringhe si trovano in questa posizione
            result_string += elemento
            result_string += " "
       
        result_string = result_string.replace(".","") #rimozione punti 
        regex = r"\((\d\w*)\)([A-Z ]+)"
        matches = re.findall(regex, result_string)
        output = []
        for match in matches:
            output.append(match[1])

    return matches

         




In [36]:
query = 'Mi cerchi le seguenti ricette: {Matches} '

In [ ]:
# Riprendiamo il codice del chatbot
wikipedia = WikipediaAPIWrapper(lang='it')
search = DuckDuckGoSearchRun()

wikipedia_tool = Tool(
    name = "Wikipedia Search",
    description="A wrapper around Wikipedia. "
        "Useful for when you need to answer general questions about "
        "people, places, companies, historical events, or other subjects. "
        "Input should be a search query.",
    func=wikipedia.run
)

search_tool = Tool(
    name = "Google Search",
    description="Search Google for recent results.",
    func=search.run
)#Capire se serve

wolframe_tool = Tool(
    name = "Wolfram Alpha",
    description="A wolfram alpha search engine. Useful for when you need to answer questions about Math, Science, Technology, Culture, Society and Everyday Life. Input should be a search query.",
    func=search.run
) #Dal punto di vista scientifico è più affidabile


request_tool = Tool(
    name = "requests",
    description="A portal to the internet. Use this when you need to get specific content from a site. Input should be a specific url, and the output will be all the text on that page.",
    func=search.run
)#Il request_tool può esplorare i siti se per esempio cerchiamo il sito del niguarda ci dice se fa determinate analisi


tools = [wikipedia_tool, search_tool,wolframe_tool,request_tool]
llm = OpenAI(temperature=0)
memory = ConversationBufferMemory(memory_key="chat_history")
# tools = load_tools(tool_names, llm=llm)

agent = initialize_agent(tools, llm, memory=memory, agent="conversational-react-description", verbose=True)
agent.run("Sei un agente AI che aiuta a dare informazioni medica. Input: informaizoni sugli ambulatori ed ospedali per ricette mediche")